In [52]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

In [53]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [54]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [55]:
#### 將features標準化,可以提高模型預測精準度
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0

In [56]:
#### Onehot encoding
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)


In [58]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
model = Sequential()
model.add(Conv2D(
    filters=32,
    kernel_size=(3,3),
    input_shape=(32,32,3),
    activation="relu",
    padding="same"))

#### 每次訓練迭代時，會隨機放棄25%的神經元，避免overfitting
model.add(Dropout(rate=0.25))

model.add(MaxPooling2D(
    pool_size=(2,2)
))

model.add(Conv2D(
    filters=64,
    kernel_size=(3,3),
    activation="relu",
    padding="same"
))
model.add(Dropout(0.25))

model.add(MaxPooling2D(pool_size=(2,2)))

#### 建立平坦層
model.add(Flatten())
model.add(Dropout(0.25))
#### 建立隱藏層
model.add(Dense(1024, activation="relu"))
model.add(Dropout(rate=0.25))
#### 建立輸出層
model.add(Dense(10, activation="softmax"))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)              0         
__________

In [60]:
#### 定義訓練方式
model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])
model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=2, 
          batch_size=200, 
          verbose=1 )


Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 299s 7ms/step - loss: 1.5191 - acc: 0.4542 - val_loss: 1.3104 - val_acc: 0.5758
Epoch 2/2
40000/40000 [==============================] - 288s 7ms/step - loss: 1.1616 - acc: 0.5871 - val_loss: 1.1694 - val_acc: 0.6327


In [61]:
prediction = model.predict_classes(x_test)

In [62]:
prediction

array([3, 8, 8, ..., 2, 4, 7], dtype=int64)

In [63]:
y_test.shape

(10000, 1)

In [64]:
y_test.reshape(-1)

array([3, 8, 8, ..., 5, 1, 7])

In [65]:
import pandas as pd
pd.crosstab(y_test.reshape(-1), 
            prediction, 
            rownames=["label"],
            colnames=["predict"])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,633,21,82,27,15,2,31,13,126,50
1,14,723,22,14,13,2,21,9,57,125
2,77,3,506,70,131,34,114,36,21,8
3,15,6,121,485,79,85,154,30,12,13
4,24,3,90,72,618,12,102,64,14,1
5,4,2,110,253,76,403,78,52,17,5
6,4,2,53,46,54,7,812,7,8,7
7,11,1,50,57,107,48,18,687,8,13
8,62,39,14,25,11,6,17,6,794,26


In [66]:
y_test  = np_utils.to_categorical(y_test)
loss,accuracy = model.evaluate(x_test, y_test)

10000/10000 [==============================] - 21s 2ms/step


In [67]:
print(loss)

1.15853500443


In [68]:
print(accuracy)

0.6324
